In [ ]:
%env OE_LICENSE=/Users/alexpayne/oe_license.txt
import numpy as np
import sys
import pickle as pkl
import importlib
import re
from openeye import oechem
oechem.OEChemIsLicensed("python")

In [ ]:
mcss_pkl = "/Users/alexpayne/Downloads/mcs_sort_index.pkl"

In [ ]:
compound_ids, xtal_ids, sort_idxs = pkl.load(open(mcss_pkl, "rb"))

In [ ]:
len(compound_ids)

# Mpro file

In [ ]:
mpro_fragalysis_sdf = "/Users/alexpayne/Downloads/Mpro_combined.sdf"

In [ ]:
ifs = oechem.oemolistream()
ifs.open(mpro_fragalysis_sdf)
mols = [mol.CreateCopy() for mol in ifs.GetOEGraphMols()]

In [ ]:
len(mols)

In [ ]:
for mol in mols:
    print(mol.GetTitle())
#     print(oechem.OEGetSDData(mol, "SMILES"))

In [ ]:
type(mol)

## Confirm that I can convert all these "compound ids" into real compounds ids

In [ ]:
import yaml

In [ ]:
cmpd_to_frag_fn = "../metadata/cmpd_to_frag.yaml"

In [ ]:
with open(cmpd_to_frag_fn) as f:
    cmpd_to_frag_dict = yaml.safe_load(f)

In [ ]:
frag_to_cmpd_dict = {v:k for k,v in cmpd_to_frag_dict.items()}

In [ ]:
frag_to_cmpd_dict

In [ ]:
succeeded = []
failed = []
for mol in mols:
    try:
        succeeded.append((mol.GetTitle(), frag_to_cmpd_dict[mol.GetTitle()[:-3]]))
    except KeyError:
        failed.append(mol.GetTitle())

In [ ]:
len(failed)

In [ ]:
len(succeeded)

In [ ]:
print([name for name in failed])

In [ ]:
succeeded[1]

# Why only 220?

In [ ]:
sys.path.append("../")
from asapdiscovery import data
importlib.reload(data)

In [ ]:
structure_dir = "/Users/alexpayne/Scientific_Projects/mers-drug-discovery/mpro-paper-ligand/aligned/"
xtal_csv = "/Users/alexpayne/Scientific_Projects/mers-drug-discovery/mpro-paper-ligand/extra_files/Mpro_compound_tracker_csv.csv"

In [ ]:
xtal_compounds = data.fragalysis.parse_xtal(xtal_csv, structure_dir)
# for xtal in xtal_compounds:
#     ## Get chain
#     ## The parentheses in this string are the capture group
#     re_pat = rf"/{xtal.dataset}_([0-9][A-Z])/"
#     try:
#         frag_chain = re.search(re_pat, xtal.str_fn).groups()[0]
#     except AttributeError:
#         print(
#             f"Regex chain search failed: {re_pat}, {xtal.str_fn}.",
#             "Using A as default.",
#             flush=True,
#         )
#         frag_chain = "0A"
#     xtal.output_name = f"{xtal.dataset}_{frag_chain}_{xtal.compound_id}"

#     ## We also want the chain in the form of a single letter ('A', 'B'), etc
#     xtal.active_site_chain = frag_chain[-1]

In [ ]:
len(xtal_compounds)

In [ ]:
data

## it is filtered to 220 in the parse_xtal function

### copied here:

In [ ]:
def parse_xtal(x_fn, x_dir, p_filter=True, dataset_filter=True):
    """
    Load all crystal structures into schema.CrystalCompoundData objects.
    Parameters
    ----------
    x_fn : str
        CSV file giving information on each crystal structure
    x_dir : str
        Path to directory containing directories with crystal structure PDB
        files
    Returns
    -------
    List[schema.CrystalCompoundData]
        List of parsed crystal structures
    """
    import pandas, os

    df = pandas.read_csv(x_fn)

    ## Filter the compound tracker
    idx = []
    for d in df["Dataset"]:
        keep = True
        if dataset_filter:
            keep = (type(d) is str)
        if p_filter:
            keep = keep and ("-P" in d)
        idx.append(keep) 
    
#     idx = [(type(d) is str) and ("-P" in d) for d in df["Dataset"]]
    print(sum(idx))

    ## Build argument dicts for the CrystalCompoundData objects
    xtal_dicts = [
        dict(zip(("smiles", "dataset", "compound_id"), r[1].values))
        for r in df.loc[idx, ["SMILES", "Dataset", "Compound ID"]].iterrows()
    ]

    ## Add structure filename information
    for d in xtal_dicts:
        fn_base = f'{x_dir}/{d["dataset"]}_0{{}}/{d["dataset"]}_0{{}}_{{}}.pdb'
        for suf in ["seqres", "bound"]:
            for chain in ["A", "B"]:
                fn = fn_base.format(chain, chain, suf)
                if os.path.isfile(fn):
                    d["str_fn"] = fn
                    break
            if os.path.isfile(fn):
                break

    ## Build CrystalCompoundData objects for each row
    xtal_compounds = [data.schema.CrystalCompoundData(**d) for d in xtal_dicts]

    return xtal_compounds

In [ ]:
xtal_compounds = parse_xtal(xtal_csv, structure_dir, False, False)

In [ ]:
print([xtal.smiles for xtal in xtal_compounds])

In [ ]:
print([xtal.compound_id for xtal in xtal_compounds])

In [ ]:
coi = [c for c in xtal_compounds if c.dataset=="Mpro-x10862"]

In [ ]:
coi

In [ ]:
coi = [c for c in xtal_compounds if c.dataset=="Mpro-x10889"]

In [ ]:
coi

# Test Making GraphMols and saving sdf from parse_xtal input

In [ ]:
from asapdiscovery.data.utils import save_openeye_sdf

## save multiple molecule sdf file

In [ ]:
def save_openeye_sdf(mol, sdf_fn):
    ofs = oechem.oemolostream()
    ofs.SetFlavor(oechem.OEFormat_SDF, oechem.OEOFlavor_SDF_Default)
    ofs.open(sdf_fn)
    oechem.OEWriteMolecule(ofs, mol)
    ofs.close()

def save_openeye_sdfs(mols, sdf_fn):
    """
    Parameters
    ----------
    mols: list of OEGraphMol
    sdf_fn: str
        SDF file path
    Returns
    -------
    list of OEGraphMol objects
    """
    ofs = oechem.oemolostream()
    ofs.SetFlavor(
        oechem.OEFormat_SDF,
        oechem.OEOFlavor_SDF_Default,
    )
    if ofs.open(sdf_fn):
        for mol in mols:
            oechem.OEWriteMolecule(ofs, mol)
        ofs.close()
    else:
        oechem.OEThrow.Fatal(f"Unable to open {sdf_fn}")
    

    return True

## make sure i'm saving the right set

In [ ]:
xtal_compounds = parse_xtal(xtal_csv, structure_dir)

## what data can we save?

In [ ]:
mol = xtal_compounds[0]

In [ ]:
mol

## make OEGraphMol for each compound

In [ ]:
mols = []
for c in xtal_compounds:
    new_mol = oechem.OEGraphMol()
    oechem.OESmilesToMol(new_mol, c.smiles)
    oechem.OEGetSDData(new_mol, f"Compound_ID")
    oechem.OESetSDData(new_mol, f"SMILES", c.smiles)
    oechem.OESetSDData(new_mol, f"Dataset", c.dataset)
    oechem.OESetSDData(new_mol, f"Compound_ID", c.compound_id)
    new_mol.SetTitle(c.compound_id)
    mols.append(new_mol)

## Save Openeye SDFs

In [ ]:
save_openeye_sdfs(mols, "/Volumes/Rohirrim/local_test/sars_docking/fragalysis_correct_bond_orders.sdf")